## Imports

In [1]:
import pandas as pd
import sqlite3 as db
import numpy as np

## Constants

In [70]:
PROJ = '/Users/dylansmith/Documents/AnalyticsInPolitics/'
DEMO_PATH = PROJ + 'data/raw/demographics/'
IND_PATH = PROJ + 'data/raw/indicators/'
PROC_PATH = PROJ + 'data/processed/'
SQL_US_ANALYSIS =  db.connect(PROC_PATH + 'us_pop_factors.db')
REPORT_PATH = PROJ + 'reports/'
EDUCATION_FIELDS = ['LESS_HS', 'HS', 'SOME_BA', 'EQ_MORE_BA']
CENSUS_FIELDS = {'Asian Alone': 'AA','American Indian Alaska': 'IA', 'White Alone':'WA','Black Alone':'BA',
                "Native Hawaiian Pacific": 'NA','Two Or More Races': 'TOM',"Hispanic": 'H'}
EMPLOYMENT_FIELDS = ['YR','FIPS','State','PctEmpAgriculture','PctEmpMining','PctEmpConstruction',
                     'PctEmpManufacturing','PctEmpTrade','PctEmpTrans','PctEmpInformation','PctEmpFIRE',
                     'PctEmpServices','PctEmpGovt','NumCivEmployed']
JOBLESS = ['YR','FIPS','State','PctEmpChange0710','PctEmpChange0718','PctEmpChange1018','PctEmpChange1718']
OUT = ['FIPS', 'STATE', 'REGION', 'RGN_DESC', 'CTY_NM', 'RURL_URBN_CD', 'URBN_INFL_CD', 'METRO_DESC', 'URBAN', 
       'URBAN_DESC', 'NEAR_METRO', 'NEAR_METRO_DESC', 'METRO', 'POP', 'POP_SIZE', 'TYPE', 'ADJACENT', 
       'ADJACENT_DESC', 'PCT_LESS_HS', 'PCT_HS', 'PCT_SOME_BA', 'PCT_EQ_MORE_BA', 'MED_HH_INC', 'PER_CAP_INC', 
       'PCT_POV_U18', 'PCT_POV_ALL', 'PCT_DEEP_POV_ALL', 'PCT_DEEP_POV_U18', 'PCT_CHG_EMPLOY_0710', 
       'PCT_CHG_EMPLOY_0718', 'PCT_CHG_EMPLOY_1018', 'PCT_CHG_EMPLOY_1718', 'AGRICULTURE', 'MINING', 
       'CONSTRUCTION', 'MANUFACTURING', 'TRADE', 'TRANSPORTATION', 'INFORMATION', 'FIRE', 'SERVICES', 
       'GOVERNMENT', 'POP_EMPLOYED']
DEMOGRAPHICS = ['AA_MALE','AA_FEMALE','IA_MALE','IA_FEMALE','WA_MALE','WA_FEMALE','BA_MALE','BA_FEMALE'
            ,'NA_MALE','NA_FEMALE','TOM_MALE','TOM_FEMALE','H_MALE','H_FEMALE']
POLL_COLS = ['FIPS','STATE','YR','OFFICE','DEMOCRAT_VOTES','OTHER_VOTES',
            'REPUBLICAN_VOTES','REPUBLICAN_PCT','DEMOCRAT','OTHER','REPUBLICAN']

## Functions

In [3]:
def readSQL(sql):
    """ Input SQL Select statement and run on the US Analysis Connection for the db connection
        ::param sql: a SQL Select statement
        returns: a Pandas Dataframe
    """
    return pd.read_sql(sql, SQL_US_ANALYSIS)

In [4]:
def executeSQL(sqlcommand, df = None):
    """ Execute SQL commands.  If the dataframe is none, no data to be inserted
        ::param sqlcommand: sql command to run on the data
        ::param df: data to be inserted into the table
    """
    if df is not None:
        data = tuple(df.itertuples(index = False))
        wildcards = ','.join(['?'] * len(list(df)))
        insert_sql = sqlcommand %  wildcards
        SQL_US_ANALYSIS.executemany(insert_sql, data)
    else:
        SQL_US_ANALYSIS.execute(sqlcommand)
    SQL_US_ANALYSIS.commit()

In [5]:
def formatCensusData(YR):
    """ Format the census data and filter out bad data
        ::param YR: Year of the census to be downloaded
        ::return -> Return a dataframe with pivoted data for the census
    """
    #download the dataframes for the age fields and the census fields
    df_full = pd.read_csv(DEMO_PATH + '2018/cc-est2018-alldata.csv',
                    index_col = False,
                    encoding='latin-1')
    df_ages = readSQL('SELECT * FROM age_codes')

    #filter out bad values and pad the string for the correct join values
    yr_val = YR - 2007
    df = df_full[df_full['YEAR'] == yr_val]
    df = df.merge(df_ages, on = 'AGEGRP', how = 'left')
    df.insert(0, 'FIPS', (df['STATE']*1000 + df['COUNTY']).astype(str).str.pad(width = 5, side = 'left', fillchar = '0'))

    df_full_pop = df[['FIPS','TOT_POP']][df['AGEGRP'] == 0]
    df_full_pop.rename(columns = {'TOT_POP':'TOT_POP_CNTY'},inplace = True)
    df = df.merge(df_full_pop, on = 'FIPS', how = 'left')

    del df_full, df_ages, df_full_pop
    #set up all of the census fields
    df = df[df['AGEGRP'] != 0]
    census_analysis = ['YR','FIPS', 'AGEGRP','AGES','TOT_POP_CNTY']

    df_val = df[['FIPS','AGES', 'AGEGRP','TOT_POP_CNTY']]
    for key, value in CENSUS_FIELDS.items():
        
        for gender in ['MALE','FEMALE']:
            demo = '%s_%s' % (value, gender)
            if value == 'WA':
                df_val.loc[:,demo] = round((df[demo] - df['H_%s' % gender]) / df['TOT_POP_CNTY'] * 100, 5)
            else:
                df_val.loc[:,demo] = round(df[demo] / df['TOT_POP_CNTY'] * 100, 3)
      
            census_analysis.append(demo)
    df_val.insert(0, 'YR', YR)
    df_val.loc[:,'AGEGRP'] = df_val['AGEGRP'].astype(int)
    df = df_val[census_analysis]
    executeSQL("DELETE FROM demographics where YR = %s" % YR)
    executeSQL("INSERT INTO demographics VALUES (%s)",  df)

In [6]:
def getGDPFactors(name):
    """ Write the data for gdp indicators into a table"""
    df = pd.read_csv(IND_PATH + '%s.csv' % name,
                     index_col = False,
                     encoding='latin-1')
    df['GeoFIPS'] = df['GeoFIPS'].str.replace('"','').str.strip()
    df['Description'] = df['Description'].str.strip()
    df.rename(columns = {'GeoFIPS': 'FIPS'}, inplace = True)
    df = df[df['FIPS'].astype(int) != 0]
    df.drop(columns = ['TableName','Region','GeoName','IndustryClassification','Unit'], inplace = True)

    for yr in range(2002,2019):
        df = df.replace({str(yr): {'(D)': None, '(NA)': None}})
        df[str(yr)] = df[str(yr)].astype('float')

    df = pd.melt(df, id_vars = ['FIPS','LineCode','Description'], value_vars = [str(YR) for YR in range(2002,2019)])
    df.rename(columns = {'variable':'YR'}, inplace = True)
    
    df.rename(columns = {'value':'USD'},inplace = True)
    df_pct = getGDPFactors('gdp_change_all_areas')
    df_pct.rename(columns = {'value':'PCT_CHG'} ,inplace = True)

    df = pd.merge(df, df_pct, how = 'left', on = ['FIPS','LineCode','YR','Description'])
    executeSQL("DELETE FROM gdp")
    executeSQL("INSERT INTO gdp VALUES (%s)",  df)

In [7]:
def aggregateAllIndicators():
    #get dataframes for all of the different demographic factors
    df_education = getIndicators("Education.xls")
    df_regions = getIndicators('regions.csv')
    df_poverty = getIndicators("PovertyEstimates.xls")
    df_unemployment = getIndicators("Unemployment.xls")

    #join urban codes as well as the
    education_fields = ['FIPS', 'COUNTY_NAME','2013_RuralUrban_Continuum_Code', '2013_Urban_Influence_Code'] + ['PCT_%s_13_17' % x for x in EDUCATION_FIELDS]
    df_education = df_education[education_fields]
    employment_fields = ['FIPS','Unemployment_rate_2016']
    poverty_fields = ['FIPS', 'PCTPOVALL_2017']
    region_fields = ['FIPS','State','REGION','DESCRIPTION']
    #join the fields together
    for (df, fields) in [(df_regions,region_fields),(df_poverty,poverty_fields),(df_unemployment,employment_fields)]:
        df_education = df_education.merge(df[fields], on = 'FIPS', how = 'left')
    df_education.insert(0, 'YR', 2016)
    return df_education[df_education['FIPS'] < '72000']

In [8]:
def getIndicators(name):
    """ Function that reads the associated indicators into the correct dataframes"""
    if '.xls' in name:
        df = pd.read_excel(IND_PATH + name,
                            index_col = False,
                            dtype = {'FIPS':np.str})
    else:
        df = pd.read_csv(IND_PATH + name,
                            index_col = False,
                            dtype = {'FIPS':np.str})
    return df

In [33]:
def groupAges(row):
    agegrp = row['AGEGRP']
    #1-5: Young & Pre College, 6-8: Young Professionals, 9-13: Boomer, 14 < : Super old
    if agegrp <=5 :
        val = 0
        desc = 'YOUNG'
    elif agegrp in [6,7,8]:
        val = 1
        desc = 'YOUNG_PROF'
    elif agegrp in [9,10,11,12,13]:
        val = 2
        desc = 'BOOMER'
    else:
        val = 3
        desc = 'OLD'
    return desc

## Run Data Processing

In [19]:
#formatAndInsertGDPFactors('gdp_usd_all_areas')
df_gdp = readSQL("SELECT * FROM gdp")
df_gdp.to_csv(REPORT_PATH + 'gdp.csv', index = False)
df_gdp.head()

,FIPS,INDUSTRY_CODE,DESCRIPTION,YEAR,USD,PCT_CHG
0,01000,1,All industry total,2002,127792310.0,NaN
1,01000,2,Private industries,2002,106583943.0,NaN
2,01000,3,"Agriculture, forestry, fishing and hunting",2002,1588009.0,NaN
3,01000,6,"Mining, quarrying, and oil and gas extraction",2002,1207862.0,NaN
4,01000,10,Utilities,2002,3442155.0,NaN


In [66]:
dfI = readSQL("SELECT * FROM indicators")
dfp = readSQL("SELECT * FROM poverty")
dfec = readSQL("SELECT * FROM employment_change")
dfe = readSQL("SELECT * FROM employment")
dfru = readSQL("SELECT * FROM rural_urban")
dfui = readSQL("SELECT * FROM urban_influence")

In [12]:
dfe.head()

,YR,FIPS,STATE,AGRICULTURE,MINING,CONSTRUCTION,MANUFACTURING,TRADE,TRANSPORTATION,INFORMATION,FIRE,SERVICES,GOVERNMENT,POP_EMPLOYED
0,2018,00000,US,1.300426,0.570714,6.350992,10.277208,14.083655,5.100678,2.107117,6.579266,48.964665,4.665278,150599165.0
1,2018,01000,AL,1.111987,0.459935,6.413107,14.187046,14.496215,5.209561,1.619647,5.602505,45.389487,5.510509,2055509.0
2,2018,01001,AL,0.605508,0.315196,5.134373,13.321168,12.881553,7.436131,1.671367,5.573988,41.908593,11.152123,24112.0
3,2018,01003,AL,1.028740,0.478068,7.608878,9.803746,16.498933,4.835413,1.376121,7.611112,45.920225,4.838764,89527.0
4,2018,01005,AL,4.223924,0.168957,5.631899,23.755350,14.665465,6.240144,0.394233,3.671998,33.318315,7.929714,8878.0


In [13]:
dfI.head()

,YR,FIPS,ST,REGION,RGN_DESC,CTY_NM,RURL_URBN_CD,URBN_INFL_CD,PCT_LESS_HS,PCT_HS,PCT_SOME_BA,PCT_EQ_MORE_BA,PCTPOVALL,UNEMPLOYMENT_RT
0,2016,01001,AL,1,DEEP SOUTH,Autauga County,2,2,12.3,33.6,29.1,25.0,13.4,5.1
1,2016,01003,AL,1,DEEP SOUTH,Baldwin County,3,2,9.8,27.8,31.7,30.7,10.1,5.3
2,2016,01005,AL,1,DEEP SOUTH,Barbour County,6,6,26.9,35.5,25.5,12.0,33.4,8.3
3,2016,01007,AL,1,DEEP SOUTH,Bibb County,1,1,17.9,43.9,25.0,13.2,20.2,6.4
4,2016,01009,AL,11,GREATER APPALACHIA,Blount County,1,1,20.2,32.3,34.4,13.1,12.8,5.4


In [14]:
dfp.head()

,YR,FIPS,STATE,MED_HH_INC,PER_CAP_INC,PCT_POV_U18,PCT_POV_ALL,PCT_DEEP_POV_ALL,PCT_DEEP_POV_U18
0,2018,00000,US,60336.0,31177.0,18.4,13.4,6.477016,8.989297
1,2018,01000,AL,48193.0,25746.0,24.4,16.9,7.842565,12.099103
2,2018,01001,AL,58343.0,27824.0,19.3,13.4,5.854320,9.668756
3,2018,01003,AL,56607.0,29364.0,14.7,10.1,5.003669,6.789429
4,2018,01005,AL,32490.0,17561.0,50.3,33.4,12.683073,26.613051


In [15]:
dfec.head()

,YR,FIPS,STATE,PCT_CHG_EMPLOY_0710,PCT_CHG_EMPLOY_0718,PCT_CHG_EMPLOY_1018,PCT_CHG_EMPLOY_1718
0,2018,01000,AL,-5.96,1.11,7.52,1.47
1,2018,01001,AL,-0.62,6.10,6.76,0.23
2,2018,01003,AL,-6.22,12.93,20.42,2.39
3,2018,01005,AL,-7.49,-18.01,-11.37,1.39
4,2018,01007,AL,-6.14,-1.36,5.09,1.75


In [16]:
dfru.head()

,CODES,METRO,METRO_DESC,POP,URBAN,URBAN_DESC,NEAR_METRO,NEAR_METRO_DESC
0,1,1,METRO,MORE_1000000,2,METRO,1,YES
1,2,1,METRO,250000_1000000,2,METRO,1,YES
2,3,1,METRO,LESS_250000,2,METRO,1,YES
3,4,0,NONMETRO,MORE_20000,1,URBAN,1,YES
4,5,0,NONMETRO,MORE_20000,1,URBAN,0,NO


In [67]:
#get all the columns together
dfI.drop(columns = ['PCTPOVALL','UNEMPLOYMENT_RT','YR'], inplace = True)
dfp.drop(columns = ['YR'], inplace = True)
dfec.drop(columns = ['YR'], inplace = True)
dfe.drop(columns = ['YR'], inplace = True)

dfI = dfI.merge(dfru, left_on = 'RURL_URBN_CD', right_on = 'CODES', how = 'left')
dfI.drop(columns = ['CODES','POP','METRO'], inplace = True)
dfI = dfI.merge(dfui, left_on = 'URBN_INFL_CD', right_on = 'CODES', how = 'left')
dfI = dfI.merge(dfp, left_on = ['FIPS','ST'], right_on = ['FIPS','STATE'], how = 'left')
dfI = dfI.merge(dfec, left_on = ['FIPS','ST'], right_on = ['FIPS','STATE'], how = 'left')
dfI = dfI.merge(dfe, left_on = ['FIPS','ST'], right_on = ['FIPS','STATE'], how = 'left')
dfI.drop(columns = ['CODES','ST','STATE_x','STATE_y'], inplace = True)

dfI[OUT].head()

,FIPS,STATE,REGION,RGN_DESC,CTY_NM,RURL_URBN_CD,URBN_INFL_CD,METRO_DESC,URBAN,URBAN_DESC,...,MINING,CONSTRUCTION,MANUFACTURING,TRADE,TRANSPORTATION,INFORMATION,FIRE,SERVICES,GOVERNMENT,POP_EMPLOYED
0,01001,AL,1,DEEP SOUTH,Autauga County,2,2,METRO,2,METRO,...,0.315196,5.134373,13.321168,12.881553,7.436131,1.671367,5.573988,41.908593,11.152123,24112.0
1,01003,AL,1,DEEP SOUTH,Baldwin County,3,2,METRO,2,METRO,...,0.478068,7.608878,9.803746,16.498933,4.835413,1.376121,7.611112,45.920225,4.838764,89527.0
2,01005,AL,1,DEEP SOUTH,Barbour County,6,6,NONMETRO,1,URBAN,...,0.168957,5.631899,23.755350,14.665465,6.240144,0.394233,3.671998,33.318315,7.929714,8878.0
3,01007,AL,1,DEEP SOUTH,Bibb County,1,1,METRO,2,METRO,...,3.953005,9.166565,20.548281,10.549504,5.947864,1.076980,4.405825,38.930363,5.005507,8171.0
4,01009,AL,11,GREATER APPALACHIA,Blount County,1,1,METRO,2,METRO,...,0.724977,10.285313,14.812909,15.528531,7.324602,1.169317,5.102900,38.264733,5.163704,21380.0


In [71]:
df_poll = readSQL("SELECT * FROM polling")

df_poll['PARTY_TWO'] = np.where((df_poll['PARTY']=='democrat') | (df_poll['PARTY']=='republican'), 
                                df_poll['PARTY'].str.upper(), 'OTHER')
df_poll['PARTY_CAND'] = np.where((df_poll['PARTY']=='democrat') | (df_poll['PARTY']=='republican'), 
                                df_poll['CANDIDATE'], 'OTHER')

df_poll.drop(columns = ['PARTY','SPECIAL','CANDIDATE','DISTRICT','TOTAL_VOTES'] ,inplace = True)
df_poll['YR'] = df_poll['YR'].astype(int)
df_total = df_poll.groupby(['YR', 'STATE','FIPS', 'OFFICE','PARTY_TWO','PARTY_CAND'])['VOTES'].sum().reset_index()

full = df_poll.pivot_table(values = 'VOTES', index = ['YR', 'STATE','FIPS', 'OFFICE']
                            ,columns = 'PARTY_TWO').replace(np.nan, 0).reset_index()
full_cand = df_poll.pivot_table(values = 'PARTY_CAND', 
                            index = ['YR', 'STATE','FIPS', 'OFFICE'], 
                            columns = 'PARTY_TWO',
                            aggfunc=lambda x: ' '.join(str(v) for v in x)).replace(np.nan, 'NAN').reset_index()
full.rename(columns = {'DEMOCRAT': 'DEMOCRAT_VOTES', 'OTHER':'OTHER_VOTES','REPUBLICAN':'REPUBLICAN_VOTES'}, inplace = True)
df_poll = pd.merge(full, full_cand, how = 'inner', on = ['YR', 'STATE','FIPS', 'OFFICE'])
df_poll['REPUBLICAN_PCT'] = df_poll['REPUBLICAN_VOTES'] / (df_poll['REPUBLICAN_VOTES'] + df_poll['OTHER_VOTES'] + df_poll['DEMOCRAT_VOTES'])
df_poll[POLL_COLS].head()

PARTY_TWO,FIPS,STATE,YR,OFFICE,DEMOCRAT_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,REPUBLICAN_PCT,DEMOCRAT,OTHER,REPUBLICAN
0,02013,AK,1976,US House,34194.0,0.0,83722.0,0.710014,Eben Hopson,NAN,Don Young
1,02016,AK,1976,US House,34194.0,0.0,83722.0,0.710014,Eben Hopson,NAN,Don Young
2,02020,AK,1976,US House,34194.0,0.0,83722.0,0.710014,Eben Hopson,NAN,Don Young
3,02050,AK,1976,US House,34194.0,0.0,83722.0,0.710014,Eben Hopson,NAN,Don Young
4,02060,AK,1976,US House,34194.0,0.0,83722.0,0.710014,Eben Hopson,NAN,Don Young


In [45]:
df_demo = readSQL("SELECT * FROM demographics")
df_demo['AGEGRP'] = df_demo.apply(groupAges, axis = 1)
df_demo.drop(columns = 'AGES', inplace = True)
df_demo = df_demo.groupby(['YR','FIPS','AGEGRP','TOT_POP_CNTY'])[DEMOGRAPHICS].sum().reset_index()

df_demo = pd.pivot_table(df_demo, values=DEMOGRAPHICS, index=['YR','FIPS','TOT_POP_CNTY'],columns = 'AGEGRP',fill_value=0).reset_index()
df_demo.columns = df_demo.columns.to_series().str.join('_').str.strip('_')
df_demo.head()

,YR,FIPS,TOT_POP_CNTY,AA_FEMALE_BOOMER,AA_FEMALE_OLD,AA_FEMALE_YOUNG,AA_FEMALE_YOUNG_PROF,AA_MALE_BOOMER,AA_MALE_OLD,AA_MALE_YOUNG,...,TOM_MALE_YOUNG,TOM_MALE_YOUNG_PROF,WA_FEMALE_BOOMER,WA_FEMALE_OLD,WA_FEMALE_YOUNG,WA_FEMALE_YOUNG_PROF,WA_MALE_BOOMER,WA_MALE_OLD,WA_MALE_YOUNG,WA_MALE_YOUNG_PROF
3142,2012,01001,54936,0.206,0.064,0.160,0.195,0.130,0.020,0.149,...,0.483,0.103,13.35190,6.22360,12.26154,7.02818,13.10797,4.88750,12.70934,6.84433
3143,2012,01003,190143,0.158,0.037,0.176,0.132,0.099,0.020,0.176,...,0.416,0.106,15.50308,8.70188,11.41299,7.01525,14.32606,7.64213,11.66018,6.74861
3144,2012,01005,27174,0.091,0.029,0.045,0.059,0.047,0.025,0.091,...,0.221,0.117,7.94877,5.51263,5.06736,2.92192,9.44651,4.60735,5.62672,4.78031
3145,2012,01007,22664,0.048,0.004,0.017,0.000,0.004,0.008,0.026,...,0.207,0.066,12.69854,6.74197,10.50564,6.53901,13.84133,5.15354,11.72785,7.49206
3146,2012,01009,57570,0.057,0.015,0.044,0.039,0.038,0.005,0.036,...,0.273,0.065,15.74605,8.48879,12.87649,7.84609,15.42817,6.63713,13.37850,7.41880


# Join all data together!

In [76]:
df_all = pd.merge(df_poll[df_poll['OFFICE'] == 'President'][POLL_COLS], df_demo, 
                      on = ['YR','FIPS'], how = 'inner')
df_all = pd.merge(df_all, dfI, on= ['FIPS','STATE'], how = 'inner')

df_all.to_csv(REPORT_PATH + 'presidential_final.csv', index = False)
df_all.head()

,FIPS,STATE,YR,OFFICE,DEMOCRAT_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,REPUBLICAN_PCT,DEMOCRAT,OTHER,...,MINING,CONSTRUCTION,MANUFACTURING,TRADE,TRANSPORTATION,INFORMATION,FIRE,SERVICES,GOVERNMENT,POP_EMPLOYED
0,01001,AL,2012,President,6363.0,190.0,17379.0,0.726183,Barack Obama,OTHER,...,0.315196,5.134373,13.321168,12.881553,7.436131,1.671367,5.573988,41.908593,11.152123,24112.0
1,01001,AL,2016,President,5936.0,865.0,18172.0,0.727666,Hillary Clinton,OTHER,...,0.315196,5.134373,13.321168,12.881553,7.436131,1.671367,5.573988,41.908593,11.152123,24112.0
2,01003,AL,2012,President,18424.0,898.0,66016.0,0.773583,Barack Obama,OTHER,...,0.478068,7.608878,9.803746,16.498933,4.835413,1.376121,7.611112,45.920225,4.838764,89527.0
3,01003,AL,2016,President,18458.0,3874.0,72883.0,0.765457,Hillary Clinton,OTHER,...,0.478068,7.608878,9.803746,16.498933,4.835413,1.376121,7.611112,45.920225,4.838764,89527.0
4,01005,AL,2012,President,5912.0,47.0,5550.0,0.482231,Barack Obama,OTHER,...,0.168957,5.631899,23.755350,14.665465,6.240144,0.394233,3.671998,33.318315,7.929714,8878.0
